# Azure Machine Learning Deployment Pipeline

 In this notebook, we will be creating a Azure Machine Learning Pipeline for the complete stage of machine learning lifecycle:
 
 1. Data Engineering
 2. Model Training
 3. Model Management
 4. Model Deployment (to same environment)
 
![Data Engineering](./images/00-Pipeline.jpg)

## Prepare for Model Package

In [1]:
import os
if not os.path.exists('model_deploy'):
    os.makedirs('model_deploy')

In [2]:
%%writefile model_deploy/score.py
import json, os, joblib
from azureml.core.model import Model

def init(): 
  global vec, clf
  print(Model.get_model_path('ProductReview-NaiveBayes'))
  vec = joblib.load(Model.get_model_path('ProductReview-CountVector'))
  clf = joblib.load(Model.get_model_path('ProductReview-NaiveBayes'))

def run(data): 
  input_data = json.loads(data)['data'] 
  fitted_data = vec.transform(input_data)
  pred = clf.predict(fitted_data)
  return json.dumps(pred.tolist())


Overwriting model_deploy/score.py


In [3]:
%%writefile model_deploy/deploy.py
from azureml.core.model import InferenceConfig, Model
from azureml.core import Environment
from azureml.core.run import Run
from azureml.core.webservice import AciWebservice

# get run context
run = Run.get_context()
workspace = run.experiment.workspace

service_name = 'product-review-service'
env = Environment.get(workspace=workspace, name="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu")

inference_config = InferenceConfig(entry_script='score.py', 
                            source_directory='.',
                            environment=env)

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

nb_model = Model(workspace, 'ProductReview-NaiveBayes')
vectorizor = Model(workspace, 'ProductReview-CountVector')

service = Model.deploy(
    workspace,
    name = service_name,
    models=[nb_model, vectorizor],
    inference_config= inference_config,
    deployment_config= deployment_config,
    overwrite=True,
)
service.wait_for_deployment(show_output=True)

Overwriting model_deploy/deploy.py


In [4]:
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import Pipeline, PipelineData
import azureml.core
from azureml.core import Workspace, Environment, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.runconfig import RunConfiguration
import os

workspace = Workspace.from_config()
# Get ComputeTarget
aml_compute_target = "cpu-cluster"
try:
    aml_compute = AmlCompute(workspace, aml_compute_target)
    print("found existing compute target.")
except ComputeTargetException:
    print("creating new compute target")
    
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2",
                                                                min_nodes = 1, 
                                                                max_nodes = 4)    
    aml_compute = ComputeTarget.create(workspace, aml_compute_target, provisioning_config)
    aml_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
print("Azure Machine Learning Compute attached")

env = Environment.get(workspace=workspace, name="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu")
# create a new runconfig object
runconfig = RunConfiguration()
runconfig.environment = env


found existing compute target.
Azure Machine Learning Compute attached


In [5]:
deploy_step   = PythonScriptStep( name="deploy_model", 
                                script_name="deploy.py", 
	                            source_directory="model_deploy", 
                                compute_target=aml_compute_target, 
                                runconfig=runconfig,
                                allow_reuse=False
	                            )
#deploy_step.run_after(select_step)

In [6]:
from azureml.pipeline.core import StepSequence
   
experiment_name = 'MLOps-Workshop'

step_sequence = StepSequence(steps=[deploy_step])
pipeline = Pipeline(workspace=workspace, steps=step_sequence)
pipeline_run = Experiment(workspace, experiment_name).submit(pipeline)

print("Pipeline is submitted for execution")

Class KubernetesCompute: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Created step deploy_model [efc77a4b][dbb75c3d-3f99-4807-b540-feca32cc9554], (This step will run and generate new outputs)
Submitted PipelineRun 3c932e68-974f-4ed4-aebf-bc6d1912ab56
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/3c932e68-974f-4ed4-aebf-bc6d1912ab56?wsid=/subscriptions/2fadeb06-9775-43ec-a256-ae5922c67d60/resourcegroups/mlops/workspaces/datamlops&tid=72f988bf-86f1-41af-91ab-2d7cd011db47
Pipeline is submitted for execution


## Test Endpoint

In [7]:
import requests, json

# replace uri below with your service endpoint
uri = 'http://a9f21b4b-7143-4c8f-8578-58fb141f3a85.westeurope.azurecontainer.io/score'

headers = {"Content-Type": "application/json"}
comments = """
"I've been using Dreamweaver (and it's predecessor Macromedia's UltraDev) for many years.  
For someone who is an experienced web designer, this course is a high-level review of the CS5 version of Dreamweaver,
 but it doesn't go into a great enough level of detail to find it very useful.\n\nOn the other hand, 
 this is a great tool for someone who is a relative novice at web design.  
 It starts off with a basic overview of HTML and continues through the concepts necessary to build a modern web site.  
 Someone who goes through this course should exit with enough knowledge to create something that does what 
 you want it do do...within reason.  Don't expect to go off and build an entire e-commerce system with only this class 
 under your belt.\n\nIt's important to note that there's a long gap from site design to actual implementation.  
 This course teaches you how to implement a design.  The user interface and overall user experience is a different 
 subject that isn't covered here...it's possible to do a great implementation of an absolutely abysmal design.  
 I speak from experience.  :)\n\nAs I said above, if you're a novice, a relative newcomer or just an experienced web 
 designer who wants a refresher course, this is a good way to do it."
"""
sample_input = json.dumps({
    'data': [comments]
})
response = requests.post(uri, data=sample_input, headers=headers)
print(response.json())

[4.0]


## Publish Pipeline

In [8]:
published_pipeline = pipeline.publish(
     name="MLOps-Workshop-Deploy-Pipeline",
     description="Published Pipeline for MLOps Workshop")


In [9]:
published_pipeline.id

'8d7fe229-d7f5-4302-b460-688023876cf5'